# Receipt Analyzer

When I moved to Australia in 2023, I became curious about the cost of groceries here. To satisfy my curiosity, I collected various receipts from different supermarkets and grocery stores. I decided to set up this notebook to automatically analyze photos of these receipts. The main purpose is to understand how much I am spending on each category of groceries and to compare the prices of different items across various supermarkets. This analysis helps in budgeting and identifying the most cost-effective stores for different types of groceries.

For this project, I followed these steps:

1. **Improve Photo Quality**
    - Enhance the quality of receipt images to improve the accuracy of text extraction using OCR (Optical Character Recognition).

2. **Extract Text from Images Using Tesseract**
    - Use Tesseract, an open-source OCR tool, to extract text from receipt images. 

    **Installation Instructions:**
    - Install Tesseract on your machine. You can download it from [Tesseract OCR](https://github.com/tesseract-ocr/tesseract).
    - For Windows, download the executable installer and follow the installation instructions.
    - For macOS, use Homebrew: `brew install tesseract`.
    - For Linux, use the package manager: `sudo apt-get install tesseract-ocr`.

    - Install the Python wrapper for Tesseract using pip:
      ```bash
      pip install pytesseract
      ```

3. **Create Supermarket and Categories JSON Files**
    - Created `supermarkets.json` and `categories.json` files to store data specific to Australian groceries. These files can be modified if necessary to include new supermarkets or categories.

4. **Use GPT-3.5 to Correct Extracted Text**
    - Employ GPT-3.5 to correct the extracted text. Using regular expressions alone would not suffice due to the wide range of data cleansing needed for different receipt formats and item descriptions.

5. **Create a DataFrame**
    - Construct a DataFrame containing the following columns:
        - supermarket_name: Supermarket Name
        - date_of_purchase: Date of Purchase
        - item: Item Name
        - weight: Weight of the item, if available
        - quantity: Quantity of items, default = 1
        - price: Total price of item(s)

6. **Categorize Each Item**
    - Attribute each item to a category, sub-category, and standardized product name using predefined categories and known supermarkets data.

7. **Correct for Unrealistic Price Ranges**
    - Identify and correct unrealistic price ranges to ensure the data's accuracy and reliability.

8. **Visualize Price Differences Between Supermarkets**
    - Create visualizations to compare the prices of items across different supermarkets, helping to identify which stores offer the best deals on specific products.

9. **Visualize the Share of Each Category and Sub-Category on Total Cost**
    - Generate visualizations showing the share of each category and sub-category in the total grocery expenditure. This helps in understanding spending patterns and identifying areas where cost-saving measures can be implemented.

## Step 1: Image Preprocessing, Text Extraction & Normalization
In this first section, I will import the receipt images, preprocess them, extract the text, and clean the data to prepare it for analysis. The goal is to create a structured dataset that accurately reflects the items purchased, their prices, and their categorization. To normalize the text using GPT-3.5, I used a prompt with a step-by-step approach and split the data into less error-prone chunks. The model returns a CSV file with both, the original item name and its correction.


In [240]:
import os
import json
import pandas as pd
import re
import pytesseract
from PIL import Image, ImageEnhance, ImageFilter
import openai
from io import StringIO

# Set up your OpenAI API key
openai.api_key = os.getenv('OPENAI_API_RECEIPT_KEY')

# Specify the path to the Tesseract executable (Tesseract must be installed!)
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Define the path to the 'data' folder containing receipt images
data_folder = 'data'

# Load categories from JSON file
with open('data/categories.json', 'r') as file:
    categories = json.load(file)

# Load known supermarkets from JSON file
with open('data/supermarkets.json', 'r') as file:
    known_supermarkets = json.load(file)['supermarkets']

# Get a list of all image files in the 'data' folder
image_files = [os.path.join(data_folder, file) for file in os.listdir(data_folder) if file.endswith(('png', 'jpg', 'jpeg', 'tiff'))]

# Function to preprocess image to improve OCR accuracy
def preprocess_image(image_path):
    image = Image.open(image_path)
    image = image.convert('L')  # Convert to grayscale
    image = image.filter(ImageFilter.SHARPEN)  # Sharpen image
    image = ImageEnhance.Contrast(image).enhance(2)  # Enhance contrast
    return image

# Function to extract text from image using pytesseract
def extract_text_from_image(image_path):
    try:
        image = preprocess_image(image_path)
        return pytesseract.image_to_string(image)
    except FileNotFoundError:
        print(f"File not found: {image_path}")
        return ""

# Function to find and standardize supermarket names
def find_supermarket_name(text):
    text_lower = text.lower()
    for key, name in known_supermarkets.items():
        if key in text_lower:
            return name
    return 'Unknown'

# Function to clean and structure the extracted data
def clean_receipt_data(receipt_text):
    items = []
    lines = receipt_text.split('\n')
    
    for line in lines:
        line = line.strip()
        if not line:
            continue
        
        parts = line.rsplit(' ', 1)
        if len(parts) == 2:
            item, price = parts
            try:
                price = float(price)
                quantity = 1  # Default quantity
                weight = None
                
                # Extract weight if available in the item name
                weight_match = re.search(r'(\d+\.?\d*)\s*(kg|g|ml)', item, re.IGNORECASE)
                if weight_match:
                    weight = float(weight_match.group(1)) if weight_match.group(2).lower() == 'kg' else float(weight_match.group(1)) / 1000
                
                items.append({
                    'item': item.strip(),
                    'weight': weight,
                    'price': price,
                    'quantity': quantity,
                })
            except ValueError:
                # If conversion to float fails, it's not a valid item-price line
                continue
    
    # Extract additional information
    date_of_purchase = None
    supermarket_name = find_supermarket_name(receipt_text)
    
    # Extract date of purchase (considering different date formats)
    date_match = re.search(r'\b(\d{2}[/-]\d{2}[/-]\d{4})\b', receipt_text)
    if date_match:
        date_of_purchase = date_match.group(1)

    return items, date_of_purchase, supermarket_name

# Function to send data to OpenAI API and get improved categorization
def correct_item_name(items):
    prompt = f"""
    Given the following list of purchased products in a supermarket or liquor store, complete and correct the product name for each item.
    Example:
        item: 'COLES FROZEN CAULIFL SOOGRAM'
        item_corrected: 'Coles Frozen Cauliflower 500GRAM'

    This is the list of items:
    {items}

    Step 1: Complete and correct the product name for each item. If you cannot correct an item, return the original item name.
    Step 2: Check that the number of returned items matches the number of input items. If an item is missing, add the corrected name to the list.
    Step 3: Return the completed and corrected product names in a CSV format with the columns 'item' and 'item_corrected', without any additional text or explanations.
    """
    
    client = openai.OpenAI()
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that assists in identifying product names on supermarket receipts."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=1500
    )
    
    return completion.choices[0].message.content.strip()

# Function to split a list into chunks of a given size
def chunk_list(lst, chunk_size):
    """Split a list into chunks of a given size."""
    for i in range(0, len(lst), chunk_size):
        yield lst[i:i+chunk_size]

# Function to normalize text by converting to lowercase, stripping whitespace, and removing punctuation
def normalize_text(text):
    """Normalize text by converting to lowercase, stripping whitespace, and removing punctuation."""
    text = text.lower().strip()
    text = re.sub(r'[^\w\s]', '', text)
    return text

# Function to categorize corrected item names
def categorize_item(item_corrected, categories):
    if pd.isna(item_corrected):
        return 'Other', 'None', 'None'

    item_lower = item_corrected.lower()
    for category, subcategories in categories.items():
        for subcategory, products in subcategories.items():
            for product, terms in products.items():
                for term in terms:
                    if term.lower() in item_lower:
                        return category, subcategory, product
    return 'Other', 'Other', 'Other'

Let's run the preprocessing, text extraction and normalization:

In [241]:
# Extract data from all receipt images
all_items = []
for image_file in image_files:
    # Extract text from the image
    receipt_text = extract_text_from_image(image_file)
    if receipt_text:
        # Clean and structure the extracted data
        items, date_of_purchase, supermarket_name = clean_receipt_data(receipt_text)
        for item in items:
            all_items.append({
                'file_path': image_file,
                'supermarket_name': supermarket_name,
                'date_of_purchase': date_of_purchase,
                'item': item['item'],
                'weight': item['weight'],
                'price': item['price'],
                'quantity': item['quantity']
            })

# Create a DataFrame
df = pd.DataFrame(all_items)

# Normalize item name
df['item'] = df['item'].apply(normalize_text)

## Step 2: Data Cleasning using GPT-3.5
Here, we remove all duplicate items from the list of items and perform a normalization of the product names using GPT-3.5. The returned CSV file is then parsed and the corrected names added to the dataframe. A check for NaN values ensures that we know how many items ran into an error when corrected.

In [242]:
# Convert df['item'] column to a list without duplicates
unique_items_list = df.drop_duplicates(subset=['item'])['item'].tolist()

# Process items in chunks of 20
chunk_size = 20

# Initialize an empty list to store DataFrames
dataframes_list = []

# Process each chunk
for chunk in chunk_list(unique_items_list, chunk_size):
    improved_data = correct_item_name(chunk)
    
    # Convert the CSV response to a DataFrame
    csv_data = StringIO(improved_data)
    df_chunk = pd.read_csv(csv_data)
    
    # Append the chunk DataFrame to the list
    dataframes_list.append(df_chunk)

# Concatenate all DataFrames in the list
df_corrected = pd.concat(dataframes_list, ignore_index=True)

# Calculate the percentage of NaN values
nan_count = df_corrected['item_corrected'].isna().sum()
total_count = len(df_corrected)
nan_percentage = (nan_count / total_count) * 100

if nan_percentage <= 5:
    # Drop NaN values
    df_corrected = df_corrected.dropna(subset=['item_corrected'])
    print(f"Dismissed {nan_count} out of {total_count} items due to NaN values.")
else:
    print(f"More than 5% of the items have NaN values. Total NaN count: {nan_count}")

Dismissed 0 out of 199 items due to NaN values.


## Step 3: Categorization of products
Now, we use the corrected item name and map it to known terms in the categories JSON. Whenever a term is not found, the item is mapped to "Other". We take a look at the items which couldn't be mapped and either add the missing terms to the categories or remove the items.

In [243]:
# Apply the categorization function to each corrected item
df_corrected[['category', 'subcategory', 'product']] = df_corrected['item_corrected'].apply(
    lambda x: pd.Series(categorize_item(x, categories))
)

# Merge the original DataFrame with the corrected DataFrame on 'item'
df_merged = pd.merge(df, df_corrected, on='item', how='left')

# Clean df of irrelevant terms (adjust accordingly)
irrelevant_terms = ['tax invoice', 'receipt', 'mascot', 'abn','phone', 'subtotal', 'broadway']
df_merged_cleaned = df_merged[~df_merged['item_corrected'].str.contains('|'.join(irrelevant_terms), case=False, na=False)]

# Remove NaN values
df_merged_cleaned = df_merged_cleaned[~df_merged_cleaned['item_corrected'].isna()]

# Let's have a look at the uncategorized items
df_merged_cleaned[df_merged_cleaned.category == "Other"].item_corrected

20                                      EFT 46
22                                   glebe now
79                            Phare 7 Marvy Al
81                                    Rogister
82                                         204
88                        Register 110 Heoalpt
146                       Date 21/01/2024 Time
155                         Axe Deodorant 45ml
182           Coles Agnolotti Ricotta 600 gram
202                          Unable to correct
213    Vallumundi Farm Organic Produce 600GRAM
222                          Unable to correct
Name: item_corrected, dtype: object

In this case, we see some items that couldn't be corrected, and mainly parts of text on the receipt which are not items and can be removed. Terms like "Ricotta" or "Deodorant" seem not to be included in the categories JSON and should be added. For the purpose of showing the process, I didn't add them here.

If we're ok to dismiss all of these items, we proceed with the next line of code:

In [244]:
# If ok to dismiss them, we remove them
df_merged_cleaned = df_merged_cleaned[df_merged_cleaned.category != "Other"]

## Step 4: Correct for Unrealistic Price Ranges
In some instances, decimal separators can not be recognized due to bad image quality. For this case, I defined a range of reasonable prices for each product category. If the price in not within this range, the reason is probably a missing decimal separator. Hence, we divide the number by 10 until it reaches a reasonable range.

In [245]:
# Define realistic price ranges for each category (example ranges, adjust as needed)
price_ranges = {
    'Baby': (0, 50),
    'Bakery': (0, 20),
    'Beverages': (0, 100),
    'Condiments': (0, 50),
    'Dairy': (0, 50),
    'Frozen': (0, 100),
    'Household': (0, 50),
    'Meat': (0, 100),
    'Other': (0, 50),
    'Pantry': (0, 50),
    'Produce': (0, 50),
    'Snacks': (0, 20)
}

# Adjust prices that are too high
def adjust_price(row):
    category = row['category']
    price = row['price']
    min_price, max_price = price_ranges.get(category, (0, float('inf')))
    
    while price > max_price:
        price /= 10
        
    return price

df_merged_cleaned['price'] = df_merged_cleaned.apply(adjust_price, axis=1)

## Step 5: Visualize Price Differences Between Supermarkets
Now, we finally get to visualize our processed data. Let's see what the prices are among products in different supermarkets.

In [246]:
# Create a pivot table
pivot_table = pd.pivot_table(
    df_merged_cleaned,
    values='price',
    index=['category', 'subcategory', 'product'],
    columns='supermarket_name',
    aggfunc='median',
    fill_value=None
)

# Replace NaN values with None and format numbers to be in $, like $9.00
pivot_table = pivot_table.applymap(lambda x: f"${x:.2f}" if pd.notnull(x) else None)

# Replace NaN values with None
pivot_table = pivot_table.where(pd.notnull(pivot_table), None)

# Display the pivot table
pivot_table

supermarket_name                               Coles Liquorland Woolworths
category   subcategory    product                                         
Baby       Food           Baby Food            $1.70       None       None
Bakery     Bread          Bread                $4.12       None       None
           Pastries       Pastry               $5.00       None       None
Beverages  Alcoholic      Beer                 $7.20      $7.00       None
           Non-Alcoholic  Coffee               $7.90       None       None
                          Juice                $0.00       None       None
                          Soda                 $5.60       None       None
Condiments Seasoning      Seasoning            $2.00       None       None
Dairy      Cheese         Cheese               $4.20       None     $15.09
           Cream          Cream                $3.05       None       None
           Milk Products  Milk                 $4.30       None       None
           Yogurt         Yogurt               $8.00       None       None
Frozen     Vegetables     Frozen Vegetables    $3.80       None       None
Household  Cleaning       Cleaning Products    $2.00       None       None
Meat       Fresh Meat     Beef                $23.00       None       None
           Processed Meat Ham                   None       None     $10.09
           Seafood        Tofu                 $2.30       None       None
Pantry     Baking         Baking Ingredients   $2.70       None       None
           Canned Goods   Canned Vegetables    $1.00       None       None
           Grains & Seeds Grains               $7.15       None       None
                          Seeds                $6.30       None       None
           Seasoning      Seasoning            $4.30       None       None
Produce    Fruits         Apple                $9.65       None       None
                          Banana               $5.37       None       None
                          Blueberry            $2.65       None       None
                          Grape                $3.00       None       None
                          Kiwi                 $4.50       None       None
                          Mango                $2.95       None       None
                          Peach                $1.30       None       None
                          Pomegranate          $3.50       None       None
                          Raspberry            $6.00       None       None
                          Strawberry           $2.00       None       None
           Vegetables     Avocado              $3.60       None       None
                          Beans                $4.95       None       None
                          Beet                 $3.50       None       None
                          Broccoli             $7.40       None       None
                          Cabbage              $3.50       None       None
                          Carrot               $3.00       None       None
                          Cauliflower          $3.40       None       None
                          Celery               $4.70       None       None
                          Cucumber             $2.75       None       None
                          Eggplant             $3.89       None       None
                          Lettuce              $4.70       None       None
                          Mushrooms            $4.00       None       None
                          Onion                $3.60       None       None
                          Pepper               $4.50       None       None
                          Potato               $3.15       None       None
                          Tomato               $3.50       None       None
Snacks     Salty          Corn Thins           $2.00       None       None
                          Crackers             $9.00       None       None
                          Other                $9.00       None      $2.01
           Sweet         

Since most of the grocery shopping was done in a Coles store, there are not many prices which can be compared among supermarkets.
## Step 6: Visualize the Share of Each Category and Sub-Category on Total Cost
Let's have a look at how much is spent on each category and sub-category for the receipts used.

In [247]:
import altair as alt

# Aggregate the total amount spent per category
category_totals = df_merged_cleaned.groupby('category')['price'].sum().reset_index()
category_totals = category_totals.rename(columns={'price': 'Total cost'})

# Calculate the percentage of total cost per category
total_cost = category_totals['Total cost'].sum()
category_totals['Percentage of Total'] = (category_totals['Total cost'] / total_cost)

# Aggregate the total amount spent per subcategory
subcategory_totals = df_merged_cleaned.groupby(['category', 'subcategory'])['price'].sum().reset_index()
subcategory_totals = subcategory_totals.rename(columns={'price': 'Total cost'})

# Calculate the percentage of total cost per subcategory
subcategory_totals['Percentage of Total'] = (subcategory_totals['Total cost'] / total_cost)

# Sort categories by total cost
category_totals = category_totals.sort_values(by='Total cost', ascending=False).reset_index(drop=True)
category_totals = category_totals.reset_index()

# Sort subcategories by total cost within each category
subcategory_totals['category'] = pd.Categorical(subcategory_totals['category'], 
                                                categories=category_totals['category'], 
                                                ordered=True)
subcategory_totals = subcategory_totals.sort_values(by=['category', 'Total cost'], ascending=[True, False]).reset_index(drop=True)
subcategory_totals = subcategory_totals.reset_index()

# Create the outer donut chart for categories
outer_chart = alt.Chart(category_totals).mark_arc(innerRadius=120, outerRadius=180).encode(
    theta=alt.Theta(field='Total cost', type='quantitative'),
    color=alt.Color(field='category', type='nominal', sort=category_totals['category'].tolist(), scale=alt.Scale(scheme='tableau20'),legend=alt.Legend(title='Category')),
    tooltip=[alt.Tooltip('category:N', title='Category'),
             alt.Tooltip('Total cost:Q', format='$,.2f'),
             alt.Tooltip('Percentage of Total:Q', format='.1%')],
    order=alt.Order(field='index', type='quantitative', sort='ascending')
).properties(
    title='Total Amount Spent in Australian Supermarkets per Category'
)

# Create the inner donut chart for subcategories
inner_chart = alt.Chart(subcategory_totals).mark_arc(innerRadius=60, outerRadius=120).encode(
    theta=alt.Theta(field='Total cost', type='quantitative'),
    color=alt.Color(field='subcategory', type='nominal', legend=None, scale=alt.Scale(scheme='tableau20')),
    tooltip=[alt.Tooltip('subcategory:N', title='Sub-category'),
             alt.Tooltip('Total cost:Q', format='$,.2f'),
             alt.Tooltip('Percentage of Total:Q', format='.1%')],
    order=alt.Order(field='index', type='quantitative', sort='ascending')
)

# Combine the charts
final_chart = alt.layer(outer_chart, inner_chart).resolve_scale(color='independent').properties(
    title='Total Amount Spent in Australian Supermarkets per Category'
).properties(
    width=400,
    height=400,
    padding={"left": 60, "right": 30, "top": 20, "bottom": 20},
    autosize=alt.AutoSizeParams(contains='padding')
)

final_chart.display()


alt.LayerChart(...)

It seems that half of the costs can be attributed to produce and beverages. Vegetables and alcoholic beverages (which are quite expensive in Australia) make up the highest cost sub-categories. Dairy is the next largest category in which milk and cheese makes up the biggest part. Meat only makes up around 6% of total cost, hinting that not much meat is consumed in this household. 

# Conclusion
Overall, it's interesting to see how prices of products fluctuate and how much of the grocery shopping is spent on which category and sub-category. It can help to identify personal consumption patterns and find out if a certain product is currently cheaper or pricier than usual. The supermarket comparison would have shed some light on whether a supermarket has consistently cheaper prices than others on certain product. But due to the lack of data, the comparison must be made with further data which will eventually be added.